In [8]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 29 21:39:37 2023

@author: Bebesi Laci
"""

import sys
import os
import re
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import sqlite3

sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
from factor_model.offline_elements.loaders.price_data_loaders import (
    generate_ytickets,
    generate_price_data_map
)
from factor_model.offline_elements.loaders.market_cap_loader import (
    generate_market_cap_data, generate_market_cap_only
)
from factor_model.offline_elements.loaders.crypto_coin_loaders import (
    get_ticker_list
)
from factor_model.offline_elements.return_generators.total_return_calculation import generate_return_data
from datetime import date, timedelta
from factor_model.offline_elements.database_generators.generate_databases import refresh_raw_price_database
# from analysis.tickers import ticker_list
# from analysis.utility import convert_str_numbs_to_float
# from analysis.styles.return_based import generate_x_month_momentum
# from analysis.market_cap import generate_market_cap_data
# from analysis.ticker_basics import generate_ytickets, generate_price_data_map
# from analysis.returns import generate_return_data
# from analysis.estimation_basis import generate_estimation_basis
# from datetime import date, timedelta
# import matplotlib.pyplot as plt
# import datetime
# import statsmodels.api as sm
# from analysis.factor_return_dataset import create_factor_return_data, create_expo_from_daily_data
# from tqdm import tqdm
# import matplotlib.pyplot as plt

DATABASE_LOCATION = r"K:/Thesis/database"
RAW_DATA_DB = "raw_price_data.db"

In [3]:

parameters = {
    "HORIZON" : "20y",
    "ESTIMATION_HORIZON": int(6.5*365),
    "WEIGHT_FUNCTION" : np.sqrt,
    "PRESENT_IN_MARKET" : 3*365,  # trade days, 3 years approx
    "ESTIMATION_DAY": date.today(),
    "MARKET_CAP_COVERAGE" : 0.82,
    "NEW_COIN_INCLUSION": 365,  # after X days
    "REGRESSORS_SET1": ["market", "size", "momentum", "reversal"],
    "REGRESSORS_SET2": ["new_coin"]
}
parameters["CORE_UNIV_START"] = parameters["ESTIMATION_DAY"] - timedelta(parameters["PRESENT_IN_MARKET"])


In [4]:
tickers = get_ticker_list()

In [5]:
yfinance_tickers = generate_ytickets(tickers)
price_data_map = generate_price_data_map(yfinance_tickers, parameters["HORIZON"])


In [6]:
market_cap_df = generate_market_cap_only(yfinance_tickers)


In [9]:
return_data_map = generate_return_data(price_data_map)

K:/Thesis/codes/crypto_project\factor_model\offline_elements\return_generators\total_return_calculation.py:10: RuntimeWarning: divide by zero encountered in true_divide
  price_data_map[key]["Close"].head(-1)
K:/Thesis/codes/crypto_project\factor_model\offline_elements\return_generators\total_return_calculation.py:10: RuntimeWarning: invalid value encountered in true_divide
  price_data_map[key]["Close"].head(-1)


In [66]:
refresh_raw_price_database(price_data_map, DATABASE_LOCATION)

k:\P375\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [58]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, "RAW_DATA_DB")) as conn:
    df = pd.read_sql_query("SELECT * FROM raw_price_data", conn)
